In [4]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [10]:
movies = pd.read_csv('../프로젝트csv/movies.csv')

In [11]:
def genre_nm_none_list(df):
    title_list=[]
    title_list.append(df.loc[df['GENRE_NM'].isna(), 'MOVIE_NM'].values)
    return title_list

In [12]:
result_list = genre_nm_none_list(movies)
movie_title = result_list[0].tolist()
genre_lists = movies['GENRE_NM'].unique()

In [13]:
# 동적 웹 크롤링으로 장르 받아오기 (naver)
def naver_crawling_genre(genre_non_list):
    dv = webdriver.Chrome()
    dv.get('http://www.naver.com')
    time.sleep(1.5)
    el = dv.find_element(By.CSS_SELECTOR, 'input#query')
    for title in genre_non_list :
        el.clear()
        el.send_keys('영화 {} 정보'.format(title))
        el.send_keys(Keys.ENTER)
        time.sleep(1.5)
        genres = dv.find_elements(By.CSS_SELECTOR, 'div.info_group dd')[2]
        el = dv.find_element(By.CSS_SELECTOR, 'input#nx_query')
        genre = genres.text.split(', ')[0]
        flag = True
        for word in list(genre_lists):
            if word==genre:
                flag = False
                break;
        if(flag):
            genre = np.nan
        movies.loc[movies['MOVIE_NM']==title, 'GENRE_NM'] = genre
        el.clear()
    dv.close()

In [14]:
naver_crawling_genre(movie_title)
movies.to_csv('../프로젝트csv/장르웹크롤링.csv', index=False, encoding='utf-8')
movies = pd.read_csv('../프로젝트csv/장르웹크롤링.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6043 entries, 0 to 6042
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NO                    6043 non-null   int64  
 1   MOVIE_NM              6043 non-null   object 
 2   DRCTR_NM              5173 non-null   object 
 3   MAKR_NM               2364 non-null   object 
 4   INCME_CMPNY_NM        3197 non-null   object 
 5   DISTB_CMPNY_NM        6043 non-null   object 
 6   OPN_DE                6041 non-null   object 
 7   MOVIE_TY_NM           6043 non-null   object 
 8   MOVIE_STLE_NM         6043 non-null   object 
 9   NLTY_NM               6043 non-null   object 
 10  TOT_SCRN_CO           5638 non-null   object 
 11  SALES_PRICE           2053 non-null   object 
 12  VIEWNG_NMPR_CO        4188 non-null   object 
 13  SEOUL_SALES_PRICE     2612 non-null   object 
 14  SEOUL_VIEWNG_NMPR_CO  4548 non-null   object 
 15  GENRE_NM             